# Misson to Mars

### 1) Mars News

In [1]:
# import Dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import re

In [2]:
# set the chromedriver path
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
news_title = soup.find("div", class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text

In [5]:
print(news_title, news_p)

The 'Claw Game' on Mars: NASA InSight Plays to Win NASA's InSight lander will be the first mission to use a robotic arm to grasp instruments from the spacecraft and place them on the surface of another planet. 


### 2) JPL Mars Space Images - Featured Image

In [6]:
space_img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(space_img_url)

In [7]:
xpath = '//*[@id="full_image"]'
img_button = browser.find_by_xpath(xpath)

In [8]:
img_button.click()

In [9]:
browser.is_element_not_present_by_css("img.fancybox-image", wait_time=1)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
featured_image_url = soup.find("img", class_="fancybox-image")["src"]

if "http:" not in featured_image_url:
    featured_image_url = "https://www.jpl.nasa.gov" + featured_image_url

In [10]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16105_ip.jpg'

### 3) Mars Weather

In [11]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_weather = soup.find("p", class_="TweetTextSize", text=re.compile("Sol")).text

### 4) Mars Facts

In [12]:
space_facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(space_facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [13]:
df = tables[0]
df.columns = ["description", "value"]
df.set_index("description", inplace=True)

In [14]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [15]:
# cleanup the table
html_table_cleaned = html_table.replace('\n', '')

###  Mars Hemispheres

In [16]:
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
xpath1 = '//*[@id="product-section"]/div[2]/div[1]/a/img'
xpath2 = '//*[@id="product-section"]/div[2]/div[2]/a/img'
xpath3 = '//*[@id="product-section"]/div[2]/div[3]/a/img'
xpath4 = '//*[@id="product-section"]/div[2]/div[4]/a/img'
xpath_list = [xpath1, xpath2, xpath3, xpath4]

In [17]:
hemisphere_image_urls = []
browser.visit(hemisphere_url)

for xpath in xpath_list:
    img_button = browser.find_by_xpath(xpath)
    img_button.click()
    browser.is_element_not_present_by_css("img.jpg", wait_time=1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h2', class_="title").text
    img_url = soup.find('div', class_="wide-image-wrapper").ul.li.a['href']
    hemisphere_image_urls.append({"title": title, "img_url": img_url})
    browser.visit(hemisphere_url)

In [18]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]